In [12]:
import typing
import io
import os

import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt

from urllib.request import urlretrieve

from PIL import Image
from torchvision import transforms

from VIT import VisionTransformer, CONFIGS
from train import valid
from utils.data_utils import get_loader

from train import train, setup, set_seed


In [ ]:
config = CONFIGS["ViT-B_16"]
model = VisionTransformer(config, num_classes=200, zero_head=False, img_size=224, vis=False, extra_attention=None)
model.load_from(np.load("pretrain_weights/imagenet21k_ViT-B_16.npz"))

# weights = torch.load("pretrain_weights/imagenet21k_ViT-B_16.npz")

# model.load_state_dict(weights)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
im = Image.open("visualization/test.JPEG")
x = transform(im)
x.size()

torch.Size([3, 224, 224])

In [14]:
class Args:
    pass

args = Args()

In [15]:
def args_set(args, dataset="cifar10", extra_attention=None, num_blocks=12, train_example_nums=512, test_example_nums=128, batch_size=16, total_steps=500, warmup_steps=50, eval_every=64, learning_rate=3e-2):
    args.extra_attention = extra_attention

    args.num_blocks = num_blocks

    args.train_example_nums = train_example_nums
    args.test_example_nums = test_example_nums
    
    args.name = dataset + '_' + extra_attention + '_' + str(num_blocks) if extra_attention != None else dataset + '_' + 'origin' + '_' + str(num_blocks)
    args.dataset = dataset
    args.model_type = "ViT-B_16"
    args.pretrained_dir = "./pretrain_weights/imagenet21k_ViT-B_16.npz"
    args.output_dir = "output"

    args.img_size = 224
    args.train_batch_size = batch_size
    args.eval_batch_size = batch_size
    args.eval_every = eval_every

    args.learning_rate = learning_rate
    args.weight_decay = 0
    args.num_steps = total_steps
    args.decay_type = "cosine"
    args.warmup_steps = warmup_steps
    args.max_grad_norm = 1.0

    args.local_rank = -1
    args.seed = 42
    args.gradient_accumulation_steps = 1
    args.fp16 = False
    args.fp16_opt_level = 'O2'
    args.loss_scale = 0

    return args




dataset = 'Tiny ImageNet' #['cifar10', 'cifar100', 'Tiny ImageNet']

# extra_attention = "MoE" #[None, 'Random', 'GAU', 'Global_SlidingWindow', 'BigBird', 'MoE', 'LowRank']
extra_attention = 'GAU'
num_blocks = 12

train_example_nums = 100000 #训练数据量

test_example_nums = 10000 #测试数据量

batch_size = 16

total_steps = 50000

warmup_steps = 500

eval_every = 500

learning_rate = 3e-2



args = args_set(args, dataset, extra_attention, num_blocks, train_example_nums, test_example_nums, batch_size, total_steps, warmup_steps, eval_every, learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.device = device

set_seed(args)
args, model = setup(args)


extra attention: GAU
num of blocks: 12
single block Parameter: 7.7M
Total Parameter: 93.1M



In [17]:
valid(args, model, get_loader(args)[1], 10)

Validating... (loss=5.29832):  10%|| 61/625 [00:14<02:15,  4.15it/s] 


KeyboardInterrupt: 